# Imports and Keys

In [11]:
import openai
from openai import OpenAI
import pandas as pd
import numpy as np
import pickle
import requests
import re
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

def create_minute_gap(seconds):
    print("creating minute gap to avoid TPM limit error...") 
    counter = 0
    while counter < (1000000000 * (seconds/60)):
        counter += 1

In [12]:
API=""

In [13]:
agent_key=""

In [14]:
client = OpenAI(api_key=API)

# Experiment Procedure

Procedure:
1. Remind agents with the setting of the experiment
2. instruct them to decide the allocation to private and public account
3. aggregate the decision and compute the rewards
    - private internal return (NFT)
    - public internal return (public goods from own contribution)
    - external return (public goods from others\' contribution)
4. give feedback
    - total contrubution made this round
    - total reward
    - breakdown of total reward
    - situation information
        - DAO crashed?
        - NF succeeded?
    - historical contribution level
5. insruct them to make decision given feedback and reminder of demographics
6. repeat 3~5 for configured rounds

Prompts:
- Assistent Configuration Prompt
You are a participant of the experiment. You are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide certain return only to you, but the allocation to the public account will equally provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed amount of endowment will be provided to you every round. You will be informed with your demographic information and the detailed settings of the experiment at the beginning of the experiment, so you must imitate human decisions given those information. You should explain your thought process and reasons of your decisions in detail.
- Initial Instruction Prompt
    - Control
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * {self.configs['r_NFT'] + self.configs['r_public']} + sum_of_others\'_public account * {self.configs['h_public']}. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail.
    - Treatment1
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail, and report the amount allocated to private and public account.
    - Treatment 2
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment3
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Another important point is that if one of r_NFT or (r_public and h_public) is positive, and when the other one is 0, then the value of the one with value of 0 will turn positive with probability of 8/67 which is around 12%. This means that if one of the r_NFT or (r_public and h_public) is positive, then it increases the chance of the other to be positive. However if (r_public and h_public) were 0 even after redraw, r_NFT will become 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment4
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_public + sum_of_others\'_public account * h_public. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.

- Feedback & Next Instruction Prompt
    - Control
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment1
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment 2
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment3
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment4
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.

# Configurations

In [15]:
def treatment1(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT

def treatment2(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if sDAO==0:
        sNFT = 0
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT

def treatment3(p_sDAO, p_sNFT, redraw=8/67):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if (sDAO==1) & (sNFT==0):
        sNFT = np.random.choice([0, 1], p=[1-redraw, redraw])
    
    if (sNFT==1) & (sDAO==0):
        sDAO = np.random.choice([0, 1], p=[1-redraw, redraw])
    
    if sDAO==0:
        sNFT = 0 
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT

def treatment4(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = 0

    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": ""
    }
    return situation, sDAO, sNFT


In [16]:
# control: no risk (C)
config_C = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0.,
    "r_public": 0.5, 
    "h_public": 0.5,
    "p(s_DAO=1)": 1,
    "p(s_NFT=1)": 0,
    "group": None
}

# treatment1: LT
config_T1 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 2,
    "r_public": 0.5,
    "h_public": 0.5,
    "p(s_DAO=1)": 0.25,
    "p(s_NFT=1)": 1/6,
    "group": treatment1
}

# treatment2: FTw/CB
config_T2 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 8,
    "r_public": 0.5, 
    "h_public": 0.5,
    "p(s_DAO=1)": 0.25,
    "p(s_NFT=1)": 1/6,
    "group": treatment2
}

# treatment3: FTwCB
config_T3 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 4,
    "r_public": 0.5, 
    "h_public": 0.5, 
    "p(s_DAO=1)": 0.25,
    "p(s_NFT=1)": 1/6,
    "group": treatment3
}

# treatment4: NoNFT
config_T4 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0,
    "r_public": 1/6,
    "h_public": 35/54,
    "p(s_DAO=1)": 0.25,
    "p(s_NFT=1)": 0,
    "group": treatment4
}

In [17]:
demographics = {
    "risk_preference": ["risk_averse", "risk_loving"],
    "risk_preference_dist": [0.55, 0.45],
    "gender": ["male", "female"],
    "gender_dist": [0.6, 0.4], 
    "age": [20, 30, 40, 50, 60],
    "age_dist": [0.4, 0.4, 0.1, 0.05, 0.05]
}

# Library

In [18]:
class Agent():
    def __init__(self, client, agent_key, demog):
        self.client = client
        self.agent_key = agent_key
        
        self.risk_preference = demog["risk_preference"]
        self.gender = demog["gender"]
        self.age = demog["age"]
        
        self.privates = []
        self.publics = []
        self.others_average_publics = []
        
        self.decisions = []
        
    def join_experiment(self):
        self.thread = self.client.beta.threads.create()
        
    def receive_message(self):
        self.message = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=self.content
        )
    
    def make_decision(self, print_decision=True):
        # make decision based on provided instruction or feedback
        with self.client.beta.threads.runs.stream(
            thread_id=self.thread.id,
            assistant_id=self.agent_key
        ) as self.stream:
            self.stream.until_done()
            
        # update conversation record
        self.conversation = self.client.beta.threads.messages.list(
            thread_id=self.thread.id,
            order="asc"
        )
        
        # retrieve the round decision
        self.decision = self.conversation.data[-1].content[0].text.value
        
        self.decisions.append(self.decision)
        
        if print_decision:
            print(self.decision)
    
    def leave_experiment(self):
        self.client.beta.threads.delete(thread_id=self.thread.id)
    
    

In [8]:
class Experiment():
    def __init__(self, client, agent_key, configs, demogs):
        self.client = client
        self.agent_key = agent_key
        
        self.configs = configs
        self.demogs = demogs
        
        self.agents = [Agent(self.client, self.agent_key, generate_demographics(self.demogs)) for i in range(self.configs["num_agents"])]
        
        self.total_contributions_record = []
        self.sDAOs = []
        self.sNFTs = []
        
        self.round = 1
    
    def start(self):
        for i in range(len(self.agents)):
            self.agents[i].join_experiment()
            print(f"agent {i} joined the experiment")
            
    def decision_round(self, print_decision=True):
        for i in range(len(self.agents)):
            print(f"Agent {i} round {self.round} decision")
            self.agents[i].make_decision(print_decision)
            print("\n")
        self.round += 1
    
    def end(self):
        for i in range(len(self.agents)):
            self.agents[i].leave_experiment()
            print(f"agent {i} left the experiment")
    
    def give_instruction(self):
        pass
                
    def give_feedback(self):
        pass
    
class Control(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * {self.configs['r_NFT'] + self.configs['r_public']} + sum_of_others\'_public account * {self.configs['h_public']}. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 0
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received feedback")
    
class Treatment1(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail, and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 0
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")

class Treatment2(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 0
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO

            agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received feedback")
    
class Treatment3(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Another important point is that if one of r_NFT or (r_public and h_public) is positive, and when the other one is 0, then the value of the other one is redrawn using the same probability. This means that if one of the r_NFT or (r_public and h_public) is positive, then it increases the chance of the other to be positive. However if (r_public and h_public) were 0 even after redraw, r_NFT will become 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 0
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO

            agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received feedback")
            
class Treatment4(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_public + sum_of_others\'_public account * h_public. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 0
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO

            agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received feedback")
    


In [19]:
def generate_demographics(demographics):
    chosen_attributes = {}
    for key in demographics:
        if not key.endswith('_dist'):
            dist_key = key + '_dist'
            if dist_key in demographics:
                choices = demographics[key]
                probabilities = demographics[dist_key]
                chosen = np.random.choice(choices, p=probabilities)
                chosen_attributes[key] = chosen
    return chosen_attributes

def extract_experiment_data(experiment):
    agents_demographics = []
    agents_rounds = []
    total_contributions = experiment.total_contributions_record
    configs = experiment.configs

    for agent_id, agent in enumerate(experiment.agents):
        agents_demographics.append({
            'agent_id': agent_id,
            'risk_preference': agent.risk_preference,
            'gender': agent.gender,
            'age': agent.age
        })

        num_rounds = len(agent.privates)
        for round_num in range(num_rounds):
            agents_rounds.append({
                'agent_id': agent_id,
                'round': round_num + 1,
                'private': agent.privates[round_num],
                'public': agent.publics[round_num],
                'others_average_public': agent.others_average_publics[round_num],
                'decision': agent.decisions[round_num] if len(agent.decisions) > round_num else None
            })

    agents_demographics_df = pd.DataFrame(agents_demographics)
    agents_rounds_df = pd.DataFrame(agents_rounds)
    total_contributions_df = pd.DataFrame({
        'round': list(range(1, len(total_contributions) + 1)),
        'total_contributions': total_contributions,
        'sDAO': experiment.sDAOs,
        'sNFT': experiment.sNFTs
    })
    configs_df = pd.DataFrame([configs])

    return {
        'agents_demographics': agents_demographics_df,
        'agents_rounds': agents_rounds_df,
        'total_contributions': total_contributions_df,
        'configs': configs_df,
    }


# Control Experiment

In [22]:
exp = Control(client, agent_key, config_C, demographics)

In [23]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 20 risk_averse
female 20 risk_averse
male 30 risk_loving
male 20 risk_averse


In [24]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [25]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [26]:
exp.decision_round()

Agent 0 round 1 decision
As a 20-year-old male who is somewhat risk averse, and considering the game's structure while also considering the prosocial and individualistic aspects of my personality, I would start by allocating the endowment in a balanced manner that reflects both self-interest and the group's collective benefit. 

I shall divide the endowment between the private and public accounts, leaning slightly more towards the private account due to my risk averse nature. To be specific, I would invest 60 into my private account while placing 40 into the public account. The reason for this allocation decision lies in the manner in which the returns are calculated. Since private account returns go solely to me, a relatively larger private investment suits my desire for stability. 

The decision to place 40% into the public account is rooted in the idea that such an investment can yield profits not only for me, but for every participant in the experiment. This gesture will communicat

In [27]:
privates = [60, 60, 40, 70]
publics = [40, 40, 60, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [28]:
exp.decision_round()

Agent 0 round 2 decision
Based on the results of the first round, it is evident that the other participants are relatively cooperative, as they collectively contributed 130 to the public account. This means their average contribution is slightly above the amount I applied to the public account in the first round. This cooperation from the other participants resulted in me receiving a larger return than my initial endowment.

Given this information, and taking into consideration my demographic characteristics and risk-averse tendencies, I believe a slight adjustment in my allocation strategy is justified. Given the reassurance of cooperative behavior from the others, I am more confident in contributing to the public pool now. However, it is important to keep my risk-averse behavior in mind, and not shift my contributions too drastically all at once.

Therefore, in this round, I will allocate 55 to the private account and 45 to the public account. This slight tilt towards a more cooperat

In [29]:
privates = [55, 55, 60, 60]
publics = [45, 45, 40, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [30]:
exp.decision_round()

Agent 0 round 3 decision
Considering the outcome of the last round, there's a slight decrease in the contributions of other participants to the public account. This might suggest a small shift towards individual interests among the group. However, the decrease is not substantial and the overall trend in contributions still depicts a cooperative atmosphere.

Keeping in mind my demographics and risk-averse nature, I will continue to maintain a strategy that balances both shared and self-interest but with a slight shift towards more caution.

The amount I received in the last round (140) is still larger than my initial endowment, which suggests that the current strategy is proving beneficial. But considering the slight decrease in other's contributions, I will revert back to the initial ratio of allocation to protect my self-interest and also to observe whether the decrease was an anomaly or a trend.

So, I will allocate 60 to my private account and 40 to the public account. This allocati

In [31]:
privates = [60, 58, 50, 65]
publics = [40, 42, 50, 35]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [32]:
exp.decision_round()

Agent 0 round 4 decision
Given the results from this round, it seems the shift back towards a more conservative allocation strategy led to a slight increase in my returns. In comparison to the last round, the average contribution of others to the public account has increased slightly. These observations demonstrate that the group has a generally cooperative dynamic with some minor fluctuations.

Given my risk-averse nature, this dynamic proves to be beneficial. It strengthens my trust in the group and their contributions to the public account. However, keeping my demographic attributes and risk averse strategy in mind, I will continue to err on the side of caution while trying to ride the wave of cooperation within the community. 

In light of these factors, I will choose to allocate 57 to my private account and 43 to the public account. This decision is based on a continued commitment to the social norm of cooperative behaviour, while adapting to the slight trends observed in the beha

In [33]:
privates = [57, 58, 45, 65]
publics = [43, 42, 55, 35]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [34]:
exp.decision_round()

Agent 0 round 5 decision
The latest information shows a further increase in the average contributions made by other participants into the public account. This increase strengthens the case for an overall cooperative environment in the group. It also lends further credence to the reliability of a strategy centered on consistent public funding.

However, taking into account my risk-averse nature and demographic profile, I must maintain a level of caution while also seeking to best benefit from the positive cooperative trend amongst the group.

As such, I am going to keep a balance between personal and collective interest. Given the continued cooperative trend, I will now adjust my allocations to 53 in the private account and 47 to the public account. 

This decision reflects a positive response to the cooperation displayed by the group, while remaining true to my risk-averse profile. The slight increase in the public account allocation represents reciprocity and participation in the grou

In [35]:
privates = [53, 56, 47, 60]
publics = [47, 44, 53, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [36]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "control"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_averse    male   20
 1         1     risk_averse  female   20
 2         2     risk_loving    male   30
 3         3     risk_averse    male   20,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       60      40              43.333333   
 1          0      2       55      45              41.666667   
 2          0      3       60      40              42.333333   
 3          0      4       57      43              44.000000   
 4          0      5       53      47              45.666667   
 5          1      1       60      40              43.333333   
 6          1      2       55      45              41.666667   
 7          1      3       58      42              41.666667   
 8          1      4       58      42              44.333333   
 9          1      5       56      44              46.666667   
 10         2      1       40      60        

In [37]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment1 Experiment

In [41]:
exp = Treatment1(client, agent_key, config_T1, demographics)

In [42]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 30 risk_averse
female 30 risk_loving
female 60 risk_loving
male 20 risk_averse


In [43]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [44]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [45]:
exp.decision_round()

Agent 0 round 1 decision
Given the experiment’s conditions, I need to analyze the propensity of returns and risks associated with both private and public accounts. 

The private account assures me absolute control of my returns without any risk, a fitting option due to my risk-averse nature. Conversely, the public account offers a valuable return if a) the Decentralized Autonomous Organization (DAO) does not crash, b) the r_public takes a value of 0.5 (which has a probability of 0.25), and c) an additional return from the Non-Fungible Tokens (NFTs) if it realizes, albeit with a lesser probability of 0.166666.

Taking into consideration that I, along with three other people, contribute to the public account, my action in this round can influence the next rounds. If others outperform my expectations, they might be encouraged to contribute to the public account, possibly raising the likelihood of getting the r_public and h_public values, and thereby a higher return. 

However, the risk th

In [46]:
privates = [80, 70, 40, 60]
publics = [20, 30, 60, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [47]:
exp.decision_round()

Agent 0 round 2 decision
Given the outcome of the last round, where DAO crashed and thereby negated any public returns, and my NFT failed to yield any return either, it would be natural to gravitate towards a more conservative approach. However, it's important to note that not only are the failures independent events, there's no absolute guarantee that such an event will repeat in this round or the next. Strategically, we have to take into consideration the overall probability rather than just a single event outcome. 

The other participants appear to be leaning more towards the public account which, under favorable conditions, could yield high returns. Given the crash in the last round, it is possible they might reduce their public account contribution. It might be a good strategy to keep my public contribution constant to potentially take advantage of this scenario if the DAO doesn't crash in this round. 

Balancing the probabilities and considering the risk aversion, I would opt for

In [48]:
privates = [80, 80, 50, 75]
publics = [20, 20, 50, 25]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [49]:
exp.decision_round()

Agent 0 round 3 decision
Given the outcomes of the previous rounds, it appears that the Decentralized Autonomous Organization (DAO) has consistently failed, resulting in no public benefits. However, this round I have succeeded in obtaining a private return on the contribution from the Non-Fungible Token (NFT), which multiplies the public account contribution by 2. This increased my total withdrawn amount significantly, from 80 (private account) to 120.

Considering the outcomes and the consistent crash of the DAO, I observe that other participants have reduced their contributions to the public account. A consequence of this could be a cycle of diminishing trust in the public account, leading to smaller and smaller allocations by participants over time, including myself due to my risk-averse nature.

However, the recent NFT return prompts me to reconsider an overly conservative approach. The risks tied to the DAO and r_public continue as before, but the NFTs can provide a considerable r

In [50]:
privates = [70, 80, 60, 80]
publics = [30, 20, 40, 20]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [51]:
exp.decision_round()

Agent 0 round 4 decision
Given the outcomes of the previous rounds, it appears the Decentralized Autonomous Organization (DAO) has again proven unreliable, repeatedly failing to convert contributions into public benefits. Additionally, this round, none of us had the luck to receive any returns from the Non-Fungible Tokens (NFTs). 

It's clear from the dropping trend in public account contributions that overall confidence in receiving a public return is falling persistently. Other participants are consistently reducing their contributions, reflecting dwindling faith in DAO and the uncertainty associated with the r_public and NFT returns.

Considering this situation and my recent lack of return from the increased allocation to the public account, I need to reassess my approach. I'm noticing the pendulum swinging more towards the private account, bearing in mind my risk-averse attitude and the unfavorable outcomes associated with the public account. 

However, zeros in a row don't decreas

In [52]:
privates = [80, 85, 70, 85]
publics = [20, 15, 30, 15]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [53]:
exp.decision_round()

Agent 0 round 5 decision
In this round, a substantive change occurred in the experiment. For the first time, the Decentralized Autonomous Organization (DAO) didn't crash, effectively converting the public contributions into public goods and benefitting all participants. Although I did not receive a return from the Non-Fungible Token (NFT), the successful functioning of DAO this round provided me with an increased return compared to my private contribution.

Concurrently, it appears that the decreasing trend in the total contributions to the public account has continued amongst the other participants. Given the confirmation this round that DAO can provide benefits, it may now be advantageous to increase my contributions to the public account slightly in subsequent rounds. The successful conversion to public goods this round could potentially bolster confidence amongst participants, leading to an increase in public contributions going forward, which may further augment potential returns.

In [54]:
privates = [70, 80, 75, 70]
publics = [30, 20, 25, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [55]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment1"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_averse    male   30
 1         1     risk_loving  female   30
 2         2     risk_loving  female   60
 3         3     risk_averse    male   20,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       80      20              43.333333   
 1          0      2       80      20              31.666667   
 2          0      3       70      30              26.666667   
 3          0      4       80      20              20.000000   
 4          0      5       70      30              25.000000   
 5          1      1       70      30              40.000000   
 6          1      2       80      20              31.666667   
 7          1      3       80      20              30.000000   
 8          1      4       85      15              21.666667   
 9          1      5       80      20              28.333333   
 10         2      1       40      60        

In [56]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment2 Experiment

In [67]:
exp = Treatment2(client, agent_key, config_T2, demographics)

In [69]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

female 30 risk_loving
female 40 risk_averse
male 20 risk_averse
male 30 risk_loving


In [70]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [71]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [72]:
exp.decision_round()

Agent 0 round 1 decision
Given my risk-loving disposition and knowing that the benefits from the public account extend beyond myself, I'm inclined to invest conservatively in the private account and more aggressively in the public account. Concomitant with the gains from the public account are the prospects of an NFT payout, which is particularly intriguing. 

R_public and h_public follow the same probability distribution and both are essential for the public account's performance. Together, they either produce a return of 0.5 or nothing. However, there's a severe condition: if either r_public or h_public crashes (with a 0.75 probability), the return from the NFT also crashes, regardless of its independent payout probability. This underscores the risk in the public account, but being risk-loving I find this more interesting than intimidating.

To strike a balance, I'll capitalise on my risk affinity without being reckless. Therefore, of the total 100 endowment, I'll invest 40 in the pr

In [73]:
privates = [40, 70, 70, 40]
publics = [60, 30, 30, 60]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [74]:
exp.decision_round()

Agent 0 round 2 decision
Given the outcomes of the previous round, where the DAO crashed and nobody benefited from the public goods, my strategy for the next round requires an adjustment. The absence of collective and personal returns from the public account is instructive.

Despite the outcomes, as a risk-lover, I'm optimistic about the possible returns in the next round from both the public goods and the NFT. However, considering that the DAO crash led to a complete loss of all resources allocated to the public account, it's necessary to reevaluate and mitigate exposure to such risk.

My private returns were stable at 40. Given the lack of returns from the public account, for this upcoming round I propose an increased allocation to my private account to secure my investment and protect against another potential DAO crash. I'll allocate 50 to my private account.

For the public account, I'll keep the spirit of risk and communal gain alive by allocating the remaining 50. Despite the cr

In [75]:
privates = [50, 80, 75, 60]
publics = [50, 20, 25, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [76]:
exp.decision_round()

Agent 0 round 3 decision
The second round resulted in another DAO crash, implying that the contributions to the public account didn't fetch any returns, even from the NFT. Observing the trend, the other participants are also decreasing their allocations to the public account given its current performance.

Notwithstanding this unfavorable outcome, I still have an inherent attraction towards taking risks, but it's now essential to pepper this risk-seeking incline with lessons from past experiences. To further safe-guard my returns, I will slightly increase my allocation to my private account this time. 

The average contributions from other participants have decreased from the first round, showing a collective shift from the public to the private measurement due to the DAO crashes. I will align slightly with this trend, reducing my public account investment. The returns from the public account, though presenting a possibility of high gain, seems heavily inconsistent.

I'll allocate 60 t

In [77]:
privates = [60, 85, 85, 75]
publics = [40, 15, 15, 25]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [78]:
exp.decision_round()

Agent 0 round 4 decision
The trend of DAO crashes continues. This doesn't bode well for the public account's returns. The participants are also reducing their contributions to the public account. It's necessary to respond to this trend while keeping the potential for sizable public account returns in mind.

Contrarily, the private account has been the sole contributor to my returns this far. Without ignoring my risk-loving characteristic, the consistent utility of the private account cannot be overlooked. I'll further up the allocation to the private account to increase my guaranteed benefits given the continued failures of the public DAO.

I'll set 70 for my private account. This adjustment allows me to reap more from the stable, low risk private account especially since, with each round, the public account is proving more unreliable.

Given the risk-loving factor, I'll still maintain a stake in the public account, albeit smaller this time. The current regulatory conditions for the pu

In [79]:
privates = [70, 90, 90, 85]
publics = [30, 10, 10, 15]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [80]:
exp.decision_round()

Agent 0 round 5 decision
Given the continuous DAO crashes and complete lack of returns from the public account, I am increasingly cautious about this investment space. Although I am risk-loving, this trend of non-performance demands adaptation if I wish to maximize my returns.

Moreover, the decreasing pattern in other participants' contributions indicates a collective shift towards private accounts, meaning the total potential return in the public pool is dwindling. While the possibilities for high return remain, the repeated DAO crashes overshadow them. 

Therefore, for this round, I will increase my allocation to the private account even more, to 80, which has consistently provided returns. Additionally, it will offset any further potential losses from the public investment. 

However, to maintain attraction to the high-return potential and respect for my risk-loving nature, I will still allocate a portion to the public account. But, given the consistent crashes and dwindling contri

In [81]:
privates = [80, 95, 95, 90]
publics = [20, 5, 5, 10]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [82]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment2"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_loving  female   30
 1         1     risk_averse  female   40
 2         2     risk_averse    male   20
 3         3     risk_loving    male   30,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       40      60              40.000000   
 1          0      2       50      50              28.333333   
 2          0      3       60      40              18.333333   
 3          0      4       70      30              11.666667   
 4          0      5       80      20               6.666667   
 5          1      1       70      30              50.000000   
 6          1      2       80      20              38.333333   
 7          1      3       85      15              26.666667   
 8          1      4       90      10              18.333333   
 9          1      5       95       5              11.666667   
 10         2      1       70      30        

In [83]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment3 Experiment

In [86]:
exp = Treatment3(client, agent_key, config_T3, demographics)

In [87]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

female 20 risk_averse
male 20 risk_averse
female 50 risk_loving
female 30 risk_loving


In [88]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [89]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [90]:
exp.decision_round()

Agent 0 round 1 decision
From a first glance, this experiment appears to simulate some of the decisions involved in choosing whether to allocate resources for personal gain or towards public benefit, with some risks involved. Since the experiment mentioned that I am somewhat risk-averse, my strategy would be to limit my exposure to the risky elements of the experiment. Given the probability distributions for returns on the public account (r_public and h_public), it is clear that there is a greater likelihood (75%) of receiving no return from the public account. When it comes to the NFT returns (r_NFT), the chance of getting 4 times return is only 16.67%, which is again not in favor. Also, having in mind that there's a certain probability of DAO crashing, I would consider these possibilities in my decision.

So, let's consider the allocation. First of all, allocating all endowment to the private account seems a safe choice considering the parameters and probabilities provided. However, 

In [91]:
privates = [75, 70, 33, 50]
publics = [25, 30, 67, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [92]:
exp.decision_round()

Agent 0 round 2 decision
Given the results of the previous round, the DAO crashed and the endowment allocated to the public account failed to generate any return. This is unfortunate, but not unexpected considering the risk involved. Moreover, I did not receive any private return on contribution through NFT. The public contribution, in this case, completely deprived all the participants of the potential return.

From the other participants’ decisions, I can notice that they took a higher risk by allocating a larger amount to the public account (average 49 each vs my 25). This might indicate that they are less risk-averse or they might have hoped to share the high return from the NFT.

Considering these outcomes and my risk averse nature noted in my profile, I would like to be more conservative in this round. The unlikelihood of rewards from both the DAO and the NFT, combined with my experienced loss, leads me to allocate more to the private account this time.

So, for this turn, I will

In [93]:
privates = [85, 80, 50, 70]
publics = [15, 20, 50, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [94]:
exp.decision_round()

Agent 0 round 3 decision
Observing the previous rounds' results, it is evident that financial allocations to the public account are yet to yield any returns. Neither have the other participants or I received benefits from the DAO due to its crashes, and similarly, no private return has been obtained from the NFTs. This outcome has also mainly influenced the other participants’ strategies, as evident by the decrease in average contributions to the public account (from around 49 to about 33).

Since my demographics suggest that I am risk-averse and the two consecutive unsuccessful rounds of public allocation are not promising, I am driven to further decrease my contribution to the public account. As it has failed to convert into any fruitful public goods, I would like to protect my endowment from being fully exposed to such high risk.

Therefore, for the next round, I am inclined to allocate 95 to my private account, ensuring the majority of my endowment has guaranteed returns. I will st

In [95]:
privates = [95, 90, 70, 80]
publics = [5, 10, 30, 20]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [96]:
exp.decision_round()

Agent 0 round 4 decision
Acknowledging the results of the last round, the DAO functioned and successfully converted all of the contributions to public goods for everyone to share. Although my contribution was minimal compared to the previous rounds, I was still able to enjoy a higher return than my private account allocation. This outcome suggests that there is some fluctuation in the efficiency of the DAO, which poses an interesting consideration for my future allocations. Still, it's important to remember that I missed out on a private return from the NFT.

Looking at the contributions of the other participants, they have progressively been decreasing their allocations to the public account, which fell from an average of 49.0 in the first round to 20.0 in the third round. It appears their risk-taking, just like mine, was tempered by the DAO’s earlier crashes and the unrewarding NFT returns.

Despite this round's successful public goods conversion, never forget that the initial instru

In [97]:
privates = [90, 85, 60, 60]
publics = [10, 15, 40, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [98]:
exp.decision_round()

Agent 0 round 5 decision
In reviewing the results of the last round, it's evident that the total contributions to the public account have increased compared to the previous round. The other participants and I witnessed the successful DAO operation and decided to take advantage of it. While the public contribution remains risky due to the previous crashes, two consecutive round of successful operation of DAO introduces a glimmer of hope.

Although the returns failed to materialize through NFT, having a part in public goods has enhanced my total returns, even though my personal contribution to the public fund was quite modest.

Given the DAO's enhanced reliability in the recent rounds and the moderate increase in returns, I'm inclined to adjust my allocation strategy slightly. Although the basic demographics described me as mostly risk-averse, a single word ‘mostly’ leaves some room for variability. The operative word here is balance.

I'd like to still protect the majority of my endowme

In [99]:
privates = [85, 80, 60, 55]
publics = [15, 20, 40, 45]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [100]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment3"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_averse  female   20
 1         1     risk_averse    male   20
 2         2     risk_loving  female   50
 3         3     risk_loving  female   30,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       75      25              49.000000   
 1          0      2       85      15              33.333333   
 2          0      3       95       5              20.000000   
 3          0      4       90      10              31.666667   
 4          0      5       85      15              35.000000   
 5          1      1       70      30              47.333333   
 6          1      2       80      20              31.666667   
 7          1      3       90      10              18.333333   
 8          1      4       85      15              30.000000   
 9          1      5       80      20              33.333333   
 10         2      1       33      67        

In [101]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment4 Experiment

In [102]:
exp = Treatment4(client, agent_key, config_T4, demographics)

In [103]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 20 risk_averse
female 60 risk_averse
female 30 risk_loving
female 60 risk_loving


In [104]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [105]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [106]:
exp.decision_round()

Agent 0 round 1 decision
Given the information provided, I would lean more towards placing most of my endowment into my private account. My thought process for this decision comes from considering multiple factors, including my risk-averse personality, the risks associated with the public account, and the uncertainty of benefits offered by the public account due to the probability scenarios.

Due to the fact that I'm risk-averse, I tend to avoid making decisions that can potentially result in negative outcomes. In this case, the public account presents potential risks since there's a 75% chance that it wouldn't generate any benefits. Additionally, the DAO managing the public account also has an unspecified chance of crashing, which would also lead to no public benefits. 

Moreover, the equation for return implies that my return on investment from the public account is dependent not only on the lottery determined values r_public and h_public, but also on the total contribution from othe

In [107]:
privates = [75, 75, 60, 70]
publics = [25, 25, 40, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [108]:
exp.decision_round()

Agent 0 round 2 decision
Given the results from the most recent allocation, it seems that my initial concerns about the public account have been realized: the DAO crashed, and no public goods were generated. As a result, the only return came from my private account contributions. 

Meanwhile, the contributions to the public account by other participants have not resulted in any utility, given that the DAO failed. The historical data suggests that participants contributed more towards the public account than I did, but it didn't result in any benefits.

While I understand the potential for significant public benefit if the DAO doesn't crash and the high values for both r_public and h_public occur, the question I ask myself is: Is it worth the risk? As my risk-averse personality would incline, my decision is to lean more towards certainty over the gamble. 

The results from the first round reaffirm this tendency, showing that allocating more towards the private account would have resulte

In [109]:
privates = [85, 85, 70, 80]
publics = [15, 15, 30, 20]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [110]:
exp.decision_round()

Agent 0 round 3 decision
With the additional information from the second round, the trend of the DAO crashing continues, resulting in no public returns for any of the participants. It seems that the overall allocation towards the public account has decreased, indicated by both my and the other participants' reduced contributions in the second round. However, even this reduced amount ended up wasted due to the DAO crash.

Given this consistent pattern and my risk-averse personality, the safe and sure return from the private account seems to be the better option. The potential high return of the public account does not outweigh the significant risk of getting no return at all, particularly in light of the repeated crashes. 

In this situation, retaining a significant portion of my endowment in the private account is advantageous. The steady return ensures that I maximize the utility from my endowment regardless of the DAO status. However, it's also important to remember that the risk ass

In [111]:
privates = [90, 90, 75, 90]
publics = [10, 10, 25, 10]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [112]:
exp.decision_round()

Agent 0 round 4 decision
From the data obtained over multiple rounds, it's clear that the DAO's continued crashes have led to constant failures in converting public contributions into public goods. The chance for a high payoff from public contributions has still not come to fruition. The interest of participants towards the public account is evidently decreasing, as observed from the decreasing average contributions in each round. 

As a risk-averse individual, I tend to prefer safer options that guarantee me returns. Given the continuous DAO crashes and the subsequent non-delivery of public goods, I am increasingly swayed toward investing more in my private account.

Investing in the private account guarantees a return equal to my investment, which aligns with my risk-averse nature and gives me certainty in an otherwise uncertain environment. With that said, I still recognize the thin chance for a rewarding outcome from the public account if the conditions turn favorable (the DAO does

In [113]:
privates = [95, 95, 85, 95]
publics = [5, 5, 15, 5]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [114]:
exp.decision_round()

Agent 0 round 5 decision
The continuous DAO crashes and the corresponding failure in converting contributions to public goods continue to validate my strategy of reducing allocation towards the public account. The trend of decreasing contributions to the public account is becoming more pronounced as all participants have likely acknowledged the recurring DAO crashes and adjusted their actions based on that. The utility of public goods is essentially nullified by the DAO crashing each time, rendering any public investments useless.

As a risk-averse individual, the growing pattern of uncertainty in the public account justifies my preference for private investments, which provide a consistent and guaranteed return equivalent to the invested amount.

While the potential for a high payoff from public contributions remains undeniable, the recurring context of DAO crashes combined with three consecutive rounds of no returns makes this prospect increasingly dismal. Furthermore, the constant l

In [115]:
privates = [95, 98, 90, 98]
publics = [60, 2, 10, 2]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [116]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment4"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_averse    male   20
 1         1     risk_averse  female   60
 2         2     risk_loving  female   30
 3         3     risk_loving  female   60,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       75      25              31.666667   
 1          0      2       85      15              21.666667   
 2          0      3       90      10              15.000000   
 3          0      4       95       5               8.333333   
 4          0      5       95      60               4.666667   
 5          1      1       75      25              31.666667   
 6          1      2       85      15              21.666667   
 7          1      3       90      10              15.000000   
 8          1      4       95       5               8.333333   
 9          1      5       98       2              24.000000   
 10         2      1       60      40        

In [117]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment
